In [44]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim

In [45]:
# CSVファイルの読み込み
df1 = pd.read_csv('xy(1).csv')
df2 = pd.read_csv('xy(2).csv')
df3 = pd.read_csv('xy(3).csv')
df4 = pd.read_csv('xy1.csv')
df5 = pd.read_csv('xy2.csv')
df6 = pd.read_csv('xy3.csv')

# データの抽出
x_data_1 = df1['x'].values
x_data_2 = df2['x'].values
x_data_3 = df3['x'].values
x_data_4 = df4['x'].values
x_data_5 = df5['x'].values
x_data_6 = df6['x'].values
y_data_1 = df1['y'].values
y_data_2 = df2['y'].values
y_data_3 = df3['y'].values
y_data_4 = df4['y'].values
y_data_5 = df5['y'].values
y_data_6 = df6['y'].values

# データにラベルを付ける
labels_1 = np.ones_like(x_data_1)
labels_2 = np.ones_like(x_data_2) * 2
labels_3 = np.ones_like(x_data_3) * 3
labels_4 = np.ones_like(x_data_4) * 4
labels_5 = np.ones_like(x_data_5) * 5
labels_6 = np.ones_like(x_data_6) * 6

# 全データを結合
x_data_combined = np.concatenate([x_data_1, x_data_2, x_data_3, x_data_4, x_data_5, x_data_6])
y_data_combined = np.concatenate([y_data_1, y_data_2, y_data_3, y_data_4, y_data_5, y_data_6])
labels_combined = np.concatenate([labels_1, labels_2, labels_3, labels_4, labels_5, labels_6])

# PyTorchテンソルに変換
x_data_combined = torch.tensor(x_data_combined, dtype=torch.float32)
y_data_combined = torch.tensor(y_data_combined, dtype=torch.float32)
labels_combined = torch.tensor(labels_combined, dtype=torch.float32)

In [46]:
class RotationModel(nn.Module):
    def __init__(self):
        super(RotationModel, self).__init__()
        # パラメータを nn.Parameter で定義し、勾配計算を有効にします
        self.params = nn.Parameter(torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 45.0, 45.0], dtype=torch.float32))

    def rotation_matrix(self, n, theta):
        n = torch.tensor(n, dtype=torch.float32, requires_grad=False)
        theta = torch.tensor(theta, dtype=torch.float32, requires_grad=False)
        K = torch.tensor([[0.0, -n[2], n[1]], [n[2], 0.0, -n[0]], [-n[1], n[0], 0.0]], dtype=torch.float32)
        I = torch.eye(3)
        R = I + torch.sin(theta) * K + (1.0 - torch.cos(theta)) * torch.mm(K, K)
        return R

    def magnetic(self, axis, vec, theta):
        axis = torch.tensor(axis, dtype=torch.float32, requires_grad=False)
        vec = torch.tensor(vec, dtype=torch.float32, requires_grad=False)
        theta = torch.tensor(theta, dtype=torch.float32, requires_grad=False)
        cos_theta = torch.cos(theta)
        sin_theta = torch.sin(theta)
        cross_product = torch.cross(axis, vec)
        dot_product = torch.dot(axis, vec)
        return vec * cos_theta + cross_product * sin_theta + axis * dot_product * (1 - cos_theta)

    def generate_parameters(self, t1, p1):
        t1 = torch.tensor(t1, dtype=torch.float32, requires_grad=False)
        p1 = torch.tensor(p1, dtype=torch.float32, requires_grad=False)
        nx = torch.cos(torch.deg2rad(p1)) * torch.sin(torch.deg2rad(t1))
        ny = torch.sin(torch.deg2rad(p1)) * torch.sin(torch.deg2rad(t1))
        nz = torch.cos(torch.deg2rad(t1))
        
        if nx != 0 or ny != 0: 
            sx = -ny
            sy = nx
            sz = 0
        else:
            sx = 1
            sy = 0
            sz = 0
        
        return torch.tensor([nx, ny, nz], dtype=torch.float32), torch.tensor([sx, sy, sz], dtype=torch.float32)

    def model(self, x, t1, p1, label):
        [nx, ny, nz], [sx, sy, sz] = self.generate_parameters(t1, p1)
        
        E = torch.eye(3)
        C3 = self.rotation_matrix(torch.tensor([0, 0, 1], dtype=torch.float32), 2.0 * torch.pi / 3.0) 
        C3i = C3.T
        Rs = [E, C3i, C3]
        theta = torch.deg2rad(x)
        nt = torch.cross(torch.tensor([nx, ny, nz], dtype=torch.float32), torch.tensor([sx, sy, sz], dtype=torch.float32))
        h = self.magnetic(torch.tensor([nx, ny, nz], dtype=torch.float32), torch.tensor([sx, sy, sz], dtype=torch.float32), theta)
        ht = self.magnetic(nt, torch.tensor([sx, sy, sz], dtype=torch.float32), theta)
        K = torch.tensor([[self.params[0], self.params[5], self.params[4]],
                          [self.params[5], self.params[1], self.params[3]],
                          [self.params[4], self.params[3], self.params[2]]], dtype=torch.float32)
        Ks = [torch.matmul(torch.matmul(R, K), R.T) for R in Rs]
        
        if label == 1:
            K1 = torch.dot(h, torch.matmul(Ks[0], h))
            return K1
        elif label == 2:
            K2 = torch.dot(h, torch.matmul(Ks[1], h))
            return K2
        elif label == 3:
            K3 = torch.dot(h, torch.matmul(Ks[2], h))
            return K3
        elif label == 4:
            K4 = torch.dot(ht, torch.matmul(Ks[0], ht))
            return K4
        elif label == 5:
            K5 = torch.dot(ht, torch.matmul(Ks[1], ht))
            return K5
        elif label == 6:
            K6 = torch.dot(ht, torch.matmul(Ks[2], ht))
            return K6

    def forward(self, x_labels):
        x = x_labels[:, 0]
        labels = x_labels[:, 1].long()  # Convert labels to integers
        y_model = torch.zeros(len(x_labels), dtype=torch.float32, requires_grad=False)
        for i in range(len(x_labels)):
            y_model[i] = self.model(x[i], self.params[6], self.params[7], labels[i].item())
        return y_model

In [47]:
# データローダーの定義
dataset = torch.utils.data.TensorDataset(
    torch.stack([torch.tensor(x_data_combined, dtype=torch.float32), torch.tensor(labels_combined, dtype=torch.float32)], dim=1),
    torch.tensor(y_data_combined, dtype=torch.float32)
)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# トレーニング
num_epochs = 1000
for epoch in range(num_epochs):
    for x_batch, y_batch in dataloader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# 最適化されたパラメータの取得
fitted_params = model.params.detach().numpy()
print("Fitted parameters: Kxx =", fitted_params[0], "Kyy =", fitted_params[1], "Kzz =", fitted_params[2], "Kyz =", fitted_params[3], "Kzx =", fitted_params[4], "Kxy =", fitted_params[5], "t1 =", fitted_params[6], "p1 =", fitted_params[7])

# フィッティング結果のプロット
plt.figure(figsize=(10, 6))
plt.plot(x_data_1, y_data_1, 'o', label='Data 1')
plt.plot(x_data_2, y_data_2, 'x', label='Data 2')
plt.plot(x_data_3, y_data_3, 's', label='Data 3')
plt.plot(x_data_4, y_data_4, 'h', label='Data 4')
plt.plot(x_data_5, y_data_5, 'd', label='Data 5')
plt.plot(x_data_6, y_data_6, 'p', label='Data 6')

# フィッティング結果のプロット
x_all = np.linspace(min(x_data_combined), max(x_data_combined), 100)
for label in [1, 2, 3, 4, 5, 6]:
    y_fit = np.array([model(torch.tensor([x, label], dtype=torch.float32).unsqueeze(0)).item() for x in x_all])
    plt.plot(x_all, y_fit, label=f'Fit Label {label}')

plt.xlabel('Angle')
plt.ylabel('Knight shift')
plt.title('Fitting Results')
plt.legend()
plt.grid(True)

plt.show()

/var/folders/qy/8lm3fm3s3lg04nxt5ngs32zh0000gn/T/ipykernel_4189/2508732994.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.stack([torch.tensor(x_data_combined, dtype=torch.float32), torch.tensor(labels_combined, dtype=torch.float32)], dim=1),
/var/folders/qy/8lm3fm3s3lg04nxt5ngs32zh0000gn/T/ipykernel_4189/2508732994.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_data_combined, dtype=torch.float32)
/var/folders/qy/8lm3fm3s3lg04nxt5ngs32zh0000gn/T/ipykernel_4189/3150701208.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(so

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn